# 1️⃣ **Описание шаблона для решения задачи.**

**Задача**: обучить несколько бустингов на 3-х фолдах, выбрать лучшие, усреднить предсказания.

**Модели, которые будем обучать:**
- `CatBoostRegressor`
- `LightGBMRegressor (goss)`
- `XGBoostRegressor (dart)`


✅ Будут выполнены:
- все дополнительные условия
- возможности фреймворков (загрузка датасетов с помощью соответствующих классов, правильная подготовка категориальных признаков, early_stopping, многопоточность)
- подбор гиперпараметров для каждой модели

👀 При желании, рекомендуется проделать следующее:
- Провести EDA (Exploratory Data Analysis) и сделать выводы на основе графики
- Провести Feature Selection
- Провести Object Selection
- Использовать scheduler или custom callbacks
- Обучить дополнительные модели


❗️❗️❗️ **P.S.**
- Данный ноутбук - далеко не единственное верное решение, воспринимайте его как помощник для вашего собственного решения или чтобы побороть страх белого листа :)

- При полном заполнении ноутбука можно получить максимум 9 баллов из 10, так как из дополнительных баллов - только балл за подбор гиперпараметров.

- При любых найденных ошибках/опечатках/непонятных моментов в коде, пишите в [чат курса](https://stepik.org/lesson/681941/step/6?unit=680724)

# 2️⃣ **Подключение необходимых библиотек и загрузка данных.**

In [2]:
!pip install catboost lightgbm xgboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostRegressor, Pool

import lightgbm as lgb
from lightgbm import Dataset, LGBMRegressor

import xgboost as xgb
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_test.csv')

In [5]:
RANDOM_STATE = 101 # Ваше любимое число :)

In [6]:
results = [] # Здесь будем хранить информацию по каждой модели

# 3️⃣ **Определим вспомогательные функции.**

In [27]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import Pool

def train_model(algorithm,
                X,
                y,
                early_stopping_rounds,
                init_params=None,
                cat_features=None,
                random_seed=2023):

    scores = []
    models = []

    # Инициализация K-Fold кросс-валидации
    kf = KFold(n_splits=3, shuffle=True, random_state=random_seed)
    print(f"========= TRAINING {algorithm.__name__} =========")

    for num_fold, (train_index, val_index) in enumerate(kf.split(X)):
        # Делим данные на обучающую и валидационную выборку
        X_train, X_eval = X.iloc[train_index], X.iloc[val_index]
        y_train, y_eval = y.iloc[train_index], y.iloc[val_index]

        # Инициализация модели
        if init_params is not None:
            model = algorithm(**init_params)
        else:
            model = algorithm()

        # Обучение CatBoost
        if algorithm.__name__ == 'CatBoostRegressor':
            train_dataset = Pool(data=X_train, label=y_train, cat_features=cat_features)
            eval_dataset = Pool(data=X_eval, label=y_eval, cat_features=cat_features)

            model.fit(train_dataset,
                      eval_set=eval_dataset,
                      verbose=0,
                      early_stopping_rounds=early_stopping_rounds)

        # Обучение LightGBM
        elif algorithm.__name__ == 'LGBMRegressor':
            model.fit(
            X_train,
            y_train,
            eval_set=[(X_eval, y_eval)],
            eval_metric='rmse',
            categorical_feature=cat_features,
            # verbose=False
          )



        # Обучение XGBoost
        elif algorithm.__name__ == 'XGBRegressor':
            train_dataset = xgb.DMatrix(data=X_train, label=y_train)
            eval_dataset = xgb.DMatrix(data=X_eval, label=y_eval)

            model = xgb.train(params=init_params,
                              dtrain=train_dataset,
                              evals=[(train_dataset, 'dtrain'), (eval_dataset, 'dtest')],
                              verbose_eval=False,
                              early_stopping_rounds=early_stopping_rounds)

            X_eval = eval_dataset  # используется ниже при предсказании

        # Предсказание и вычисление RMSE
        if algorithm.__name__ == 'XGBRegressor':
            y_pred = model.predict(X_eval)
        else:
            y_pred = model.predict(X_eval)

        score = np.sqrt(mean_squared_error(y_eval, y_pred))

        models.append(model)
        scores.append(score)

        print(f'FOLD {num_fold}: RMSE = {score:.4f}')

    # Финальная метрика: среднее - стандартное отклонение
    mean_kfold_score = np.mean(scores, dtype="float16") - np.std(scores, dtype="float16")
    print("\nMEAN RMSE SCORE (mean - std):", mean_kfold_score)

    # Выбор лучшей модели
    best_model_index = np.argmin(scores)
    best_model = models[best_model_index]

    return mean_kfold_score, best_model


In [8]:
def tuning_hyperparams(algorithm,
                       X,
                       y,
                       init_params,
                       fit_params,
                       grid_params,
                       n_iter,
                       cv=3,
                       random_state=2023,
    ):

    estimator = algorithm(**init_params)

    # Можно использоавть GridSearchCV
    model = RandomizedSearchCV(estimator=estimator,
                               param_distributions=grid_params,
                               n_iter=n_iter,
                               cv=cv,
                               scoring='neg_root_mean_squared_error',
                               n_jobs=-1,
                               verbose=0,
                               random_state=random_state
    )

    model.fit(X, y, **fit_params)

    return model.best_params_ | init_params

# 4️⃣ **Группируем признаки, отбираем категориальные, выделяем датасет для обучения.**

In [10]:
# Целевые переменные — одна для регрессии, другая для классификации
targets = ['target_reg', 'target_class']

# Столбец, который нужно исключить (идентификатор)
features2drop = ['car_id']

# Категориальные признаки (по смыслу, они имеют тип 'object')
cat_features = ['model', 'car_type', 'fuel_type']

# Признаки для обучения (исключаем целевые и car_id)
filtered_features = [col for col in train.columns if col not in targets + features2drop]

# Числовые признаки — всё остальное из filtered_features
num_features = [col for col in filtered_features if col not in cat_features]

# Выводим для проверки
print("cat_features:", cat_features)
print("num_features:", num_features)
print("targets:", targets)


cat_features: ['model', 'car_type', 'fuel_type']
num_features: ['car_rating', 'year_to_start', 'riders', 'year_to_work', 'mean_rating', 'distance_sum', 'rating_min', 'speed_max', 'user_ride_quality_median', 'deviation_normal_count', 'user_uniq']
targets: ['target_reg', 'target_class']


In [11]:
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["target_reg"]

# 5️⃣ **CatBoostRegressor.**



## **Обучение модели.**

In [12]:
from catboost import CatBoostRegressor

# Инициализация гиперпараметров CatBoost для задачи регрессии
cb_init_params = {
    'loss_function': 'RMSE',           # Целевая функция — RMSE (корень из среднеквадратичной ошибки)
    'eval_metric': 'RMSE',             # Метрика для оценки на валидации
    'thread_count': -1,                # Использовать все доступные ядра
    'task_type': 'CPU',                # Используем CPU (можно поменять на 'GPU', если доступно)
    'random_seed': RANDOM_STATE        # Для воспроизводимости
}

# Обучение модели CatBoost с кросс-валидацией
cb_score, cb_model = train_model(
    algorithm=CatBoostRegressor,       # Класс модели
    X=X,                               # Признаки
    y=y,                               # Целевая переменная
    init_params=cb_init_params,        # Параметры модели
    early_stopping_rounds=50,          # Остановка, если 50 итераций без улучшения
    cat_features=cat_features,         # Список категориальных признаков
    random_seed=RANDOM_STATE           # Для воспроизводимости в KFold
)


========= TRAINING CatBoostRegressor =========
FOLD 0: RMSE = 11.9451
FOLD 1: RMSE = 11.2357
FOLD 2: RMSE = 11.6702

MEAN RMSE SCORE (mean - std): 11.33


Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [13]:
# Предсказания модели CatBoost на тестовом наборе
cb_test_pred = cb_model.predict(test[filtered_features])


pd.DataFrame({'car_id': test['car_id'], 'target_reg': cb_test_pred}).to_csv('cb_pred.csv', index=False)

In [14]:
# Если leaderboard_score пока неизвестен — используем просто kfold_score
results.append({
    'model_name': 'CatBoostRegressor',
    'tuning': False,
    'kfold_score': cb_score,
    'leaderboard_score': cb_score,  # временно используем ту же метрику
    'model': cb_model
})


## **Подбор гиперпараметров и обучение модели с новыми параметрами.**

In [15]:
# Параметры для обучения модели (fit-параметры)
cb_fit_params = {
    'cat_features': cat_features,     # Категориальные признаки
    'verbose': 0,                     # Не печатать логи
    'early_stopping_rounds': 50      # Раннее завершение обучения
}

# Сетка гиперпараметров для перебора
cb_grid_params = {
    'depth': [4, 6, 8],               # Глубина деревьев
    'learning_rate': [0.01, 0.05, 0.1],  # Скорость обучения
    'l2_leaf_reg': [1, 3, 5],         # Регуляризация
    'bagging_temperature': [0, 1, 3], # Стохастичность
    'iterations': [200, 300, 500]     # Кол-во итераций (бустов)
}

# Запуск гиперпараметрического поиска
catboost_params_after_tuning = tuning_hyperparams(
    algorithm=CatBoostRegressor,     # Модель
    X=X, y=y,                         # Данные
    init_params=cb_init_params,      # Базовые параметры
    fit_params=cb_fit_params,        # Параметры .fit()
    grid_params=cb_grid_params,      # Сетка гиперпараметров
    n_iter=20,                        # Кол-во итераций поиска (для RandomizedSearchCV)
    cv=3,                             # Кол-во фолдов в кросс-валидации
    random_state=RANDOM_STATE        # Для воспроизводимости
)

# Выводим лучшие параметры
catboost_params_after_tuning


{'learning_rate': 0.05,
 'l2_leaf_reg': 1,
 'iterations': 300,
 'depth': 6,
 'bagging_temperature': 0,
 'loss_function': 'RMSE',
 'eval_metric': 'RMSE',
 'thread_count': -1,
 'task_type': 'CPU',
 'random_seed': 101}

In [16]:
# Обучение CatBoost с лучшими подобранными гиперпараметрами
cb_tuning_score, cb_tuning_model = train_model(
    algorithm=CatBoostRegressor,                  # Используем CatBoostRegressor
    X=X,                                          # Признаки
    y=y,                                          # Целевая переменная
    early_stopping_rounds=50,                    # Раннее завершение при отсутствии улучшения
    init_params=catboost_params_after_tuning,     # Лучшие параметры после гиперпараметрического поиска
    cat_features=cat_features,                   # Список категориальных признаков
    random_seed=RANDOM_STATE                      # Для воспроизводимости
)


========= TRAINING CatBoostRegressor =========
FOLD 0: RMSE = 11.8975
FOLD 1: RMSE = 11.2961
FOLD 2: RMSE = 11.5059

MEAN RMSE SCORE (mean - std): 11.31


Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [17]:
# Предсказания на тестовом наборе с использованием модели после подбора гиперпараметров
tuning_cb_test_pred = cb_tuning_model.predict(test[filtered_features])

pd.DataFrame({'car_id': test['car_id'], 'target_reg': tuning_cb_test_pred}).to_csv('tuning_cb_pred.csv', index=False)

In [18]:
# Временно используем средний KFold score как оценку для leaderboard'а
results.append({
    'model_name': 'CatBoostRegressor',
    'tuning': True,                                 # Модель обучена с гиперпараметрическим подбором
    'mean_kfold_score': cb_tuning_score,            # Средний результат по кросс-валидации
    'leaderboard_score': cb_tuning_score,           # Пока что используем ту же метрику
    'model': cb_tuning_model                        # Сохраняем обученную модель
})


# 6️⃣ **LightGBMRegressor (goss).**

## **Подготовка категориальных признаков.**

[Ссылка](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.2%20-%20LightGBM.ipynb), если забыли, как готовить категориальные признаки

In [35]:
from sklearn.preprocessing import LabelEncoder

X_lgb = X.copy()
label_encoders = {}  # Словарь для хранения энкодеров


for col in cat_features:
    le = LabelEncoder()
    X_lgb[col] = le.fit_transform(X_lgb[col])
    label_encoders[col] = le  # сохраняем энкодер


## **Обучение модели.**

In [36]:
from lightgbm import LGBMRegressor

# Инициализация параметров для LGBMRegressor
lgb_init_params = {
    'boosting_type': 'gbdt',            # Классический градиентный бустинг
    'n_jobs': -1,                       # Использовать все ядра
    'metric': 'rmse',                   # Метрика RMSE (root mean squared error)
    'objective': 'regression',          # Задача — регрессия
    'random_state': RANDOM_STATE,       # Для воспроизводимости
    'verbosity': -1,                    # Подавить вывод логов
    'device': 'cpu'                     # Обучение на CPU (в Colab можно сменить на 'gpu')
}

# Обучение модели LGBMRegressor на закодированных данных
lgb_score, lgb_model = train_model(
    algorithm=LGBMRegressor,            # Используем scikit-learn обёртку LightGBM
    X=X_lgb,                            # Признаки (категориальные — закодированы через LabelEncoder)
    y=y,                                # Целевая переменная
    init_params=lgb_init_params,        # Параметры модели
    early_stopping_rounds=50,           # Раннее завершение при отсутствии улучшения
    cat_features=cat_features,          # Названия категориальных признаков (в формате column name)
    random_seed=RANDOM_STATE            # Для KFold и модели
)


========= TRAINING LGBMRegressor =========
FOLD 0: RMSE = 12.7004
FOLD 1: RMSE = 11.5985
FOLD 2: RMSE = 12.0731

MEAN RMSE SCORE (mean - std): 11.67


Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [37]:
# Подготовка тестовых данных для LightGBM (как и train — с LabelEncoding)
# Подготовка тестовых данных с использованием сохранённых энкодеров
X_lgb_test = test[filtered_features].copy()
for col in cat_features:
    X_lgb_test[col] = label_encoders[col].transform(X_lgb_test[col])  # <-- правильный доступ


# Предсказания модели LightGBM на тестовом наборе
lgb_test_pred = lgb_model.predict(X_lgb_test)

# Сохраняем результат в CSV
pd.DataFrame({'car_id': test['car_id'], 'target_reg': lgb_test_pred}).to_csv('lgb_pred.csv', index=False)


In [38]:
results.append({
    'model_name': 'LGBMRegressor (goss)',      # Название модели с типом бустинга
    'tuning': False,                           # Без подбора гиперпараметров
    'mean_kfold_score': lgb_score,             # Средний RMSE по кросс-валидации
    'leaderboard_score': lgb_score,            # Пока используем то же значение
    'model': lgb_model                         # Сохраняем обученную модель
})


## **Подбор гиперпараметров и обучение модели с новыми параметрами**

In [39]:
# Параметры, передаваемые в .fit() — важно указать категориальные признаки и метрику
lgb_fit_params = {
    'eval_metric': 'rmse',                  # Оценка качества (RMSE)
    'categorical_feature': cat_features     # Категориальные признаки (в LabelEncoded виде)
}

# Сетка параметров для гиперпараметрического поиска
lgb_grid_params = {
    'max_depth': [4, 6, 8],                 # Максимальная глубина дерева
    'min_data_in_leaf': [10, 20, 50],       # Минимум объектов в листе
    'learning_rate': [0.01, 0.05, 0.1],     # Скорость обучения
    'n_estimators': [200, 500, 1000],       # Кол-во итераций
    'boosting_type': ['gbdt', 'goss'],      # Тип бустинга
    'subsample': [0.7, 0.9, 1.0],           # Подвыборка (не применяется к GOSS)
    'colsample_bytree': [0.7, 0.9, 1.0]     # Подвыборка признаков
}

# Поиск лучших гиперпараметров с кросс-валидацией
lgb_params_after_tuning = tuning_hyperparams(
    algorithm=LGBMRegressor,
    X=X_lgb, y=y,
    init_params=lgb_init_params,     # Базовые параметры (например, random_state)
    fit_params=lgb_fit_params,       # Параметры .fit()
    grid_params=lgb_grid_params,     # Сетка перебора
    n_iter=20,                       # Кол-во случайных комбинаций для RandomizedSearchCV
    cv=3,                            # Кол-во фолдов для KFold
    random_state=RANDOM_STATE
)

# Показываем лучшие параметры
lgb_params_after_tuning


{'subsample': 1.0,
 'n_estimators': 500,
 'min_data_in_leaf': 20,
 'max_depth': 6,
 'learning_rate': 0.01,
 'colsample_bytree': 0.9,
 'boosting_type': 'gbdt',
 'n_jobs': -1,
 'metric': 'rmse',
 'objective': 'regression',
 'random_state': 101,
 'verbosity': -1,
 'device': 'cpu'}

In [40]:
# Обучение модели LGBMRegressor с подобранными гиперпараметрами
lgb_tuning_score, lgb_tuning_model = train_model(
    algorithm=LGBMRegressor,               # sklearn API LGBMRegressor
    X=X_lgb,                               # Данные с LabelEncoded категориальными признаками
    y=y,                                   # Целевая переменная
    init_params=lgb_params_after_tuning,   # Лучшие параметры после RandomizedSearchCV
    early_stopping_rounds=50,              # Остановка при отсутствии улучшений
    cat_features=cat_features,             # Список категориальных признаков
    random_seed=RANDOM_STATE               # Для воспроизводимости
)


========= TRAINING LGBMRegressor =========
FOLD 0: RMSE = 12.2344
FOLD 1: RMSE = 11.4121
FOLD 2: RMSE = 11.7350

MEAN RMSE SCORE (mean - std): 11.45


Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [41]:
# Добавляем модель LightGBM (с бустингом GOSS и тюнингом) в список результатов
results.append({
    'model_name': 'LGBMRegressor (goss)',
    'tuning': True,                             # Использовался подбор гиперпараметров
    'mean_kfold_score': lgb_tuning_score,       # Средняя метрика по KFold
    'leaderboard_score': lgb_tuning_score,      # Пока используем ту же метрику
    'model': lgb_tuning_model                   # Сохраняем модель
})


# 7️⃣ **XGBoostRegressor (dart).**

## **Подготовка категориальных признаков.**

[Ссылка](https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/4.3%20-%20XGBoost.ipynb), если забыли, как готовить категориальные признаки

In [42]:
from sklearn.preprocessing import LabelEncoder

# Создаём копию обучающих данных
X_xgb = X.copy()

# Применяем Label Encoding к категориальным признакам
xgb_label_encoders = {}  # словарь для сохранения энкодеров, если нужно для test

for col in cat_features:
    le = LabelEncoder()
    X_xgb[col] = le.fit_transform(X_xgb[col])
    xgb_label_encoders[col] = le  # сохраняем энкодер для использования на test


## **Обучение модели.**

In [43]:
from xgboost import XGBRegressor

xgb_init_params = {
    'enable_categorical': False,            # Мы уже закодировали категориальные признаки вручную
    'booster': 'dart',                      # Используем бустинг с Dropout
    'objective': 'reg:squarederror',        # Задача — регрессия
    'eval_metric': 'rmse',                  # Метрика RMSE
    'random_state': RANDOM_STATE,           # Для воспроизводимости
    'n_jobs': -1,                           # Использовать все потоки
    'verbosity': 0,                         # Без лишнего вывода

    # Обязательные параметры для работы dart-режима
    'rate_drop': 0.1,                       # Вероятность дропаута деревьев
    'skip_drop': 0.5                        # Вероятность пропустить dropout шаг
}

xgb_score, xgb_model = train_model(
    algorithm=XGBRegressor,                # Используем XGBoost через sklearn API
    X=X_xgb,                               # Признаки с label encoding
    y=y,                                   # Целевая переменная
    init_params=xgb_init_params,           # Параметры модели
    early_stopping_rounds=50,              # Раннее завершение при отсутствии улучшения
    cat_features=cat_features,            # Передаётся, но XGBoost их уже не использует
    random_seed=RANDOM_STATE
)


========= TRAINING XGBRegressor =========
FOLD 0: RMSE = 12.3936
FOLD 1: RMSE = 11.5977
FOLD 2: RMSE = 11.6914

MEAN RMSE SCORE (mean - std): 11.55


Сделаем предсказание для тестовой части и проверим скор на [лидерборде](https://stepik.org/lesson/779920/step/5?unit=782494)

In [45]:
# Подготовка теста
X_xgb_test = test[filtered_features].copy()
for col in cat_features:
    X_xgb_test[col] = xgb_label_encoders[col].transform(X_xgb_test[col])

# Преобразуем в DMatrix и делаем предсказания
dtest = xgb.DMatrix(X_xgb_test)
xgb_test_pred = xgb_model.predict(dtest)

# Сохраняем предсказания
pd.DataFrame({'car_id': test['car_id'], 'target_reg': xgb_test_pred}).to_csv('xgb_pred.csv', index=False)



In [46]:
results.append({
    'model_name': 'XGBRegressor (dart)',     # Модель XGBoost с бустингом DART
    'tuning': False,                         # Гиперпараметры не подбирались
    'mean_kfold_score': xgb_score,           # Средняя RMSE по кросс-валидации
    'leaderboard_score': xgb_score,          # Пока что используем ту же метрику
    'model': xgb_model                       # Сохраняем обученную модель
})


## **Подбор гиперпараметров и обучение модели с новыми параметрами**

In [47]:
# Параметры для подбора гиперпараметров XGBoost
xgb_grid_params = {
    'max_depth': [4, 6, 8],               # Максимальная глубина дерева
    'max_leaves': [10, 20, 30],           # Максимальное число листьев
    'learning_rate': [0.01, 0.05, 0.1],   # Скорость обучения
    'n_estimators': [200, 500, 1000],     # Кол-во деревьев
    'subsample': [0.7, 0.9, 1.0],         # Подвыборка объектов
    'colsample_bytree': [0.7, 0.9, 1.0],  # Подвыборка признаков
    'rate_drop': [0.1, 0.3, 0.5],         # Параметры специфичные для dart
    'skip_drop': [0.1, 0.3, 0.5]
}

# Параметры .fit() (если используешь sklearn API)
xgb_fit_params = {
    'verbose': False
}

# Поиск лучших параметров с кросс-валидацией
xgb_params_after_tuning = tuning_hyperparams(
    algorithm=XGBRegressor,            # sklearn обёртка XGBoost
    X=X_xgb, y=y,
    init_params=xgb_init_params,       # Базовые параметры, включая booster='dart'
    fit_params=xgb_fit_params,         # Передаются в .fit()
    grid_params=xgb_grid_params,       # Сетка перебора
    n_iter=20,                         # Кол-во случайных сочетаний
    cv=3,                              # Кол-во фолдов для KFold
    random_state=RANDOM_STATE
)

# Смотрим лучшие параметры
xgb_params_after_tuning


{'subsample': 0.9,
 'skip_drop': 0.5,
 'rate_drop': 0.1,
 'n_estimators': 200,
 'max_leaves': 30,
 'max_depth': 8,
 'learning_rate': 0.05,
 'colsample_bytree': 1.0,
 'enable_categorical': False,
 'booster': 'dart',
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'random_state': 101,
 'n_jobs': -1,
 'verbosity': 0}

In [48]:
# Обучение XGBRegressor (dart) с лучшими гиперпараметрами после RandomizedSearchCV
xgb_tuning_score, xgb_tuning_model = train_model(
    algorithm=XGBRegressor,                   # Используем XGBoost через sklearn API
    X=X_xgb,                                  # Признаки (с LabelEncoding)
    y=y,                                      # Целевая переменная
    init_params=xgb_params_after_tuning,      # Подобранные параметры
    early_stopping_rounds=50,                 # Остановка при отсутствии улучшения
    cat_features=cat_features,                # Не используется XGBoost'ом напрямую (но нужно для совместимости)
    random_seed=RANDOM_STATE
)


========= TRAINING XGBRegressor =========
FOLD 0: RMSE = 15.0543
FOLD 1: RMSE = 14.1114
FOLD 2: RMSE = 14.3386

MEAN RMSE SCORE (mean - std): 14.1


In [ ]:
# Подготовка тестовых данных (с LabelEncoding, как у X_xgb)
X_xgb_test = test[filtered_features].copy()
for col in cat_features:
    X_xgb_test[col] = xgb_label_encoders[col].transform(X_xgb_test[col])

# Предсказания модели XGBRegressor после тюнинга
tuning_xgb_test_pred = xgb_tuning_model.predict(X_xgb_test)

# Сохраняем результат в файл
pd.DataFrame({'car_id': test['car_id'], 'target_reg': tuning_xgb_test_pred}).to_csv('tuning_xgb_pred.csv', index=False)


In [49]:
# Добавляем модель XGBRegressor (dart) с тюнингом в список результатов
results.append({
    'model_name': 'XGBRegressor (dart)',        # Название модели
    'tuning': True,                              # Использовался подбор гиперпараметров
    'mean_kfold_score': xgb_tuning_score,        # Средняя RMSE по кросс-валидации
    'leaderboard_score': xgb_tuning_score,       # Пока используем ту же метрику
    'model': xgb_tuning_model                    # Сохраняем модель
})


# 8️⃣ **Финальное предсказание и сохранение лучших моделей**

In [52]:
# Универсальная функция для извлечения score
def get_score(entry):
    return entry.get('mean_kfold_score') or entry.get('kfold_score') or 1e9

# Лучшая CatBoost модель
best_cb_model = min(
    [r for r in results if 'CatBoost' in r['model_name']],
    key=get_score
)['model']
best_cb_model.save_model('best_cb_model.cbm')

# Лучшая LGBM модель
best_lgb_model = min(
    [r for r in results if 'LGBM' in r['model_name']],
    key=get_score
)['model']
best_lgb_model.booster_.save_model('best_lgb_model.mod')

# Лучшая XGBoost модель
best_xgb_model = min(
    [r for r in results if 'XGB' in r['model_name']],
    key=get_score
)['model']
best_xgb_model.save_model('best_xgb_model.json')


In [54]:
import xgboost as xgb

# --- Label Encoding теста для LGBM ---
X_lgb_test = test[filtered_features].copy()
for col in cat_features:
    X_lgb_test[col] = label_encoders[col].transform(X_lgb_test[col])

# --- Label Encoding теста для XGBoost ---
X_xgb_test = test[filtered_features].copy()
for col in cat_features:
    X_xgb_test[col] = xgb_label_encoders[col].transform(X_xgb_test[col])

# --- Предсказания от CatBoost ---
final_cb_pred = best_cb_model.predict(test[filtered_features])

# --- Предсказания от LightGBM ---
final_lgb_pred = best_lgb_model.predict(X_lgb_test)

# --- Предсказания от XGBoost (нужно обернуть в DMatrix) ---
dtest = xgb.DMatrix(X_xgb_test)
final_xgb_pred = best_xgb_model.predict(dtest)

# --- Ансамбль (усреднение предсказаний трёх моделей) ---
final_pred = (final_cb_pred + final_lgb_pred + final_xgb_pred) / 3

# --- Сохранение в CSV ---
pd.DataFrame({
    'car_id': test['car_id'],
    'target_reg': final_pred
}).to_csv('final_submission.csv', index=False)


# 9️⃣ **Выводы.**


In [55]:
results = pd.DataFrame(results)
results

,model_name,tuning,kfold_score,leaderboard_score,model,mean_kfold_score
0,CatBoostRegressor,False,11.328125,11.328125,<catboost.core.CatBoostRegressor object at 0x7...,NaN
1,CatBoostRegressor,True,NaN,11.312500,<catboost.core.CatBoostRegressor object at 0x7...,11.312500
2,LGBMRegressor (goss),False,NaN,11.671875,"LGBMRegressor(device='cpu', metric='rmse', n_j...",11.671875
3,LGBMRegressor (goss),True,NaN,11.453125,"LGBMRegressor(colsample_bytree=0.9, device='cp...",11.453125
4,XGBRegressor (dart),False,NaN,11.546875,<xgboost.core.Booster object at 0x7c69ae33b950>,11.546875
5,XGBRegressor (dart),True,NaN,14.101562,<xgboost.core.Booster object at 0x7c69e3f5c650>,14.101562


Примеры вопросов, на которые можно ответить при формулировании вывода:

- Какая модель показала лучшее качество на валидации/лидерборде?
- Помог ли тюнинг гиперпараметров?
- Помог ли Feature Selection?
- Помог ли Object Selection?
- Что поняли благодаря построенным графикам?
- Улучшилось ли качество на лидерборде после усреднения прогнозов моделей?
- ...

